<a href="https://colab.research.google.com/github/joaopedropastura/keras/blob/main/model_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras
!pip install pandas
!pip install tensorflow


In [4]:
import keras
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from keras import activations
from keras import initializers
from keras import regularizers
from keras import optimizers
from keras import metrics
from keras import losses
from keras import callbacks
from keras.preprocessing import image

In [ ]:
model = models.Sequencial()

In [ ]:
model.add(layers.Conv2D(
    32, (5,5), # 32 a quantidade de filtros, (5x5) = tamanho do filtro gerando imagens 60x60
    input_shape = (64, 64, 3), # 3 = 3 canais rgb, 64 = iamgens com o tamanho 64x64
    activation = 'relu'
))

model.add(layers.MaxPooling2D(
    pool_size = (2, 2) #subsalpling, vamos de 2 em 2 pegando o maior valor
))

model.add(layers.Conv2D(
    16, (5,5), # 16 a quantidade de filtros, (5x5) = tamanho do filtro gerando imagens 26x26
    input_shape = (30, 30, 3), # 3 = 3 canais rgb, 30 = iamgens com o tamanho 30x30
    activation = 'relu'
))

model.add(layers.MaxPooling2D(
    pool_size = (2, 2) #subsalpling, vamos de 2 em 2 pegando o maior valor
))

# model.add(layers.Conv2D(
#     8, (5,5), # 32 a quantidade de filtros, (5x5) = tamanho do filtro gerando imagens 60x60
#     input_shape = (13, 13, 3), # 3 = 3 canais rgb, 64 = iamgens com o tamanho 64x64
#     activation = 'relu'
# ))

# model.add(layers.MaxPooling2D(
#     pool_size = (2, 2) #subsalpling, vamos de 2 em 2 pegando o maior valor
# ))

model.add(layers.Conv2D(
    4, (5,5), # 32 a quantidade de filtros, (5x5) = tamanho do filtro gerando imagens 60x60
    input_shape = (13, 13, 3), # 3 = 3 canais rgb, 64 = iamgens com o tamanho 64x64
    activation = 'relu'
))

model.add(layers.MaxPooling2D(
    pool_size = (2, 2) #subsalpling, vamos de 2 em 2 pegando o maior valor
))


In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(
    256, #
    kernel_regularizer= regularizers.L2(1e-4),
    kernel_initializer= initializers.RandomNormal(stddev = 1), #incializadores iniciando com valores maiores que um
    bias_initializer= initializers.Zeros()
))
model.add(layers.Dropout(0.2)) #20% da camada anterior é desligada para evitar overfitting
model.add(layers.Activation(activations.relu))


model.add(layers.Dense(
    64, #
    kernel_regularizer= regularizers.L2(1e-4),
    kernel_initializer= initializers.RandomNormal(stddev = 1), #incializadores iniciando com valores maiores que um
    bias_initializer= initializers.Zeros()
))
model.add(layers.Activation(activations.relu))
#depois da camada de ativação

model.add(layers.BatchNormalization())


model.add(layers.Dense(
    64, #
    kernel_regularizer= regularizers.L2(1e-4),
    kernel_initializer= initializers.RandomNormal(stddev = 1), #incializadores iniciando com valores maiores que um
    bias_initializer= initializers.Zeros()
))
model.add(layers.Activation(activations.relu))

model.add(layers.Dense(
    2, #
    kernel_regularizer= regularizers.L2(1e-4),
    kernel_initializer= initializers.RandomNormal(stddev = 1), #incializadores iniciando com valores maiores que um
    bias_initializer= initializers.Zeros()
))
model.add(layers.Activation(activations.softmax))


In [ ]:
model.compile(
    optimizer=optimizers.Adam(),
    loss=losses.BinaryCrossentropy(),
    metrics=[metrics.Accuracy()]
)

In [ ]:
dataGen = image.ImageDataGenerator(
    rescale = 1.0/255, # transforma pixeis de ate 255 rgb para valores de 0 a 1
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    validation_split=0.2
)


X_train = dataGen.flow_from_directory(
    '/content/images',
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical', # 3(categorical, binary, regression)
    subset='training'

)

X_test = dataGen.flow_from_directory(
    '/content/images',
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical', # 3(categorical, binary, regression)
    subset='validation'

)


model.fit(
    X_train,
    steps_per_epoch = 1000,
    ephocs = 50,
    validation_steps = 100,
    callbacks= [
        callbacks.EarlyStopping(patience = 4),
        callbacks.ModelCheckpoints(
            filepath = 'model.{epoch:02d}-{val_loss:.2f}.h5'
        )
    ]
)